## Lab 2
### UGBA 88: Data and Decisions, Fall 2019

<br>

This lab is designed to be completed in class. However, in case you need additional time, this assignment is due **Tuesday, October 8th at 11:59pm**.

The lab will be graded for **completion**. Lab office hours are held by Connector Assistants on Tuesdays after labs from 2-4pm in the DS Nexus in Moffitt.

## Interference and Attrition in the Work From Home Experiment

<img src="shanghai_office.png" alt="Drawing" style="width: 400px;"/>
<center>Ctrip's Shanghai office</center>

In this lab we will reexamine the data from the Ctrip Work From Home experiment covered in Problem Set 3. We will investigate whether the baseline results from the experiment are contaminated by interference or attrition, and how these potential issues may change our interpretation of the results.

---

### Table of Contents

1. [Experiment Review](#experiment)<br>
2. [Control Group Has Lower Productivity](#control)<br>
3. [Is there an Interference Problem?](#interference)<br>
4. [Employee Attrition](#attrition)<br>
5. [Who Quits? Or: Can We Sign the Attrition Bias?](#bias)<br>


**Dependencies:**

In [ ]:
from datascience import *
import numpy as np

#These lines set up graphing capabilities.
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from client.api.notebook import Notebook
ok = Notebook('lab2.ok')
_ = ok.auth(inline=True) 

## Section 1: Experiment Review <a id='experiment'></a>

To begin we'll review the main results from the Work From Home experiment that you analyzed in Problem Set 3.

Ctrip is considering allowing their call center employees to Work From Home (WFH). Allowing employees to WFH will reduce office expenses and potentially increase employee retention by making Ctrip a more attractive place to work. However, Ctrip is worried that employees may not work as hard at home, reducing productivity. To better understand what impact this policy has on productivity, Ctrip decides to run a randomized experiment.

The site for the experiment is Ctrip's airfare and hotel departments in Shanghai. These departments take calls from customers and make bookings. Employees typically work 5 shifts a week in teams of about 15 people, plus a manager.

To recruit participants in the experiment, Ctrip asks 996 employees if they want to work from home 4 days a week. 508 volunteered. To participate in the experiment, Ctrip also requires that an employee meet additional requirements, including having their own room, broadband internet, and 6+ months of experience in the position. The final experiment group consists of 255 employees.

The **treatment group** works 4 shifts a week at home and 1 shift a week in the office, for 9 months.

The **control group** works all 5 shifts in the office.

Otherwise treatment and control groups are identical: they have the same shift, same equipment, same work-flow, and same pay structure.

To assign employees to the treatment and control groups, Ctrip holds a lottery. After a drawing, employees with even birthdays are assigned to the WFH treatment, while those with odd birthdays are assigned to the control group, and work in the office as before.

**First, we'll read in the data from the experiment. Run the cell below.**

In [ ]:
#run this cell
experiment = Table.read_table('wfh_interference.csv')
experiment.show(5)

The column `expgroup` indexes groups of employees. `expgroup = 0` corresponds to the experiment control group, and `expgroup = 1` corresponds to the treatment group. Note that we now have two extra groups of workers, `expgroup = 2` and `expgroup = 3`. We will discuss these groups later. For now, concentrate on the first two groups, treatment and control.

----

## Section 2: Control Group Has Lower Productivity <a id='control'></a>

We will revisit how worker performance compares in the treatment and control groups. While in Problem Set 3 you compared worker productivity during the experiment, here we will compare productivity both before and during the experiment. The column `performance` measures productivity during the experiment, and the column `performance_pre` measures performance during the 46 weeks prior to the experiment.

As before, `performance` and `performance_pre` summarize a variety of performance metrics, including the number of phone calls an employee answers, the number of orders taken, and the number of reservation phone calls made. However, here we will use a different scaling than what we used in the problem set. `performance_pre` is scaled to have mean 0 and standard deviation 1 across employees, but `performance` is measured using same scale. Hence, `performance` will no longer necessarily have mean 0 and standard deviation 1; in fact, you will see that the mean of `performance` is now negative, because employee productivity declines over this period.

**Q2.1** Make a bar chart that plots the averages of `performance_pre` and `performance` for the treatment and control groups. In total, your figure should include four bars.

You may want to first create a table with the average values of `performance_pre` and `performance` grouped by `expgroup`, and limit to the treatment and control groups.

In [ ]:
#bar chart of baseline experiment period performance by treatment group

#create table that takes experiment table, limits to treatment and control groups, and calculates means of performance_pre and performance
treatment_vs_control = ...

#create bar plot using above table
...

At baseline, productivity in the treatment and control groups are quite similar. This is what we should expect under random assignment.

However, during the experiment period, *large differences* in productivity emerge. *Under our standard potential outcomes assumptions*, **the difference in `performance` between the treatment and control groups reflects the *causal* effect of working from home on productivity. Our estimate for the causal effect of WFH is that it increases average performance by about 0.35 standard deviations.** This corresponds to about 15% more calls handled per day.

----

## Section 3: Is There an Interference Problem? <a id='interference'></a>

The pattern of results is somewhat odd, as that difference of 0.35 standard deviations is driven by a *decrease* in productivity in the control group.

This indicates that, for whatever reason, average productivity was lower in the control group during the experiment period than in the baseline period. This might reflect that the experiment happened to coincide with a down period for the company. By contrast, `peformance` in the treatment group is mostly unchanged.

But could it also reflect interference? Perhaps rather than having a *positive* effect on employees in the treatment group, the experiment actually had a *negative* effect on employees in the control group. The fact that some employees were allowed to work from home may have made office workers *less productive.* This would be an example of *interference*. The outcomes of some employees are affected by the treatment status of *other* employees. This complicates our interpretation of the experiment results. The difference in productivity between the treatment and control groups no longer reflects the *causal effect of WFH* per se. It's now a combination of the causal effect of WFH and the potentially negative spillover effects from having co-workers that WFH.

There are a few reasons to think this kind of interference may be an issue. First, the lottery results may have had a negative effect on the morale of employees that lost the lottery. This decline in morale may have reduced productivity in the control group. Second, the experiment may have been disruptive to operations in the Shanghai office.

Fortunately, there are two other groups of employees that can help us identify whether interference is an issue. These two groups of employees will serve as an extra set of 'control' groups. They are not subject to the potential interference issues described above. If we see large reductions in productivity for these employees, similar to what we see in the control group, that would strongly suggest the reduction is not due to inteference. However, if we see no change in productivity for these groups (or even an increase), that would suggest interference is an important issue.

The first group (with `expgroup = 2`) consists of employees in the same Shanghai office that did not originally volunteer to participate in the experiment, but would have been eligible to participate. This group should help with evaluating the first concern, since these employees did not participate in the lottery and so should not feel as though they 'lost' anything. 

The second group (with `expgroup = 3`) refers to employees in a second call center in Nan Tong who also would have met the eligibility criteria. These employees are doing the same work as the Shanghai office employees, but would not experienced any potential disruption that the WFH experiment caused for the Shanghai office.

The only problem is that employees were not *randomly assigned* to the group of non-volunteers or to the Nan Tong office. In other words, these employees may differ in systematic ways from those participating in the experiment.

Let's examine this by comparing the characteristics of employees across the four groups.

**Q3.1:** Calculate column means separately by group, this time for all four groups.

In [ ]:
#write your code here
all_groups = ...
all_groups

Unfortunately, we do not have much data on the Nan Tong employees. But we can at least compare the non-volunteers to the experiment treatment and control groups. Non-volunteers are less productive in the pre-period and have shorter commute times.

Next, let's focus on productivity. How does productivity change over time for the non-volunteers and Nan Tong employees?

**Q3.2:** Recreate the bar plot above of `performance_pre` and `performance`, but this time including all 4 groups.

In [ ]:
#bar chart of baseline experiment period performance by treatment group
...

**Question 3.3:** What do you see? Do the results make you more or less concerned about interference?

*write answer here*

We'll discuss this finding more as a group in class.

----
## Section 4: Employee Attrition <a id='attrition'></a>

Hiring and training new employees is one of the largest labor costs Ctrip faces. Executives hoped that one benefit of allowing employees to work from home is that it would reduce turnover. If WFH makes a job more desireable to employees, they may be less likely to leave. Moreover, if WFH makes employees more productive, there is less need to fire them for cause.

However, as we discussed during lecture, attrition also complicates our interpretation of the experiment results.

To this point, we've worked with data where one row represented one employee. These data are actually a summary of richer, *weekly* level data the company has on employee performance. As it turns out, it will be more instructive to use these weekly data to investigate attrition.

Run the cell below to load the weekly productivity data.

In [ ]:
#run this cell
weekly = Table.read_table('wfh_weekly.csv')
weekly.show(5)

Here's what each column denotes:
- `date`: the Monday of the work week the row refers to
- `week`: week number relative to the experiment start time. For example, `week` $= 0$ refers to the first week of the experiment; `week` $= -10$ refers to 10 weeks prior to the experiment start.

The data are limited to employees in the experiment (`exgroup = 0` and `exgroup = 1`). The data also exclude weeks where the employee was not working.

Let's first get a feel for these new data.

**Q4.1:** How many unique weeks of data do the data contain? How many weeks before the experiment, and how many after? 

*Hint:* one approach is to look at the range of values `week` takes.

In [ ]:
#write your code here
num_weeks = ...

In [ ]:
_ = ok.grade("q4_1")

**Q4.2** Next, we'll plot productivity week by week, separately by `expgroup`. We've provided most of the code for you, you just need to fill in a few blanks.

In [ ]:
# create tables for average performance by week, one for treatment group and one for control group

treatment_perform = weekly.where(...).select('week', 'performance').group('...', collect = np.mean)
control_perform = weekly.where(...).select('week', 'performance').group('...', collect = np.mean)

# generates plot

plt.plot(treatment_perform.column('...'), treatment_perform.column('performance mean'), label = 'Treatment')
plt.plot(control_perform.column('...'), control_perform.column('performance mean'), label = 'Control')
plt.title('Productivity Over Time')
plt.xlabel('Weeks Relative to Experiment Start Date')
plt.axvline(x=0)
plt.legend();

In [ ]:
_ = ok.grade("q4_2")

Note the change in pattern beginning at week 0, the start of the experiment.

So what about attrition? Does WFH make employees less likely to leave? One rough way to measure this is to look at the total number of employees working in a given week, by treatment status. This does not measure quits exactly, but it will give us some sense of how the composition of the treatment and control groups are evolving over time. Employees will leave in general, so we should expect both groups to diminish in size over time. But if one group is shrinking faster than the other, that would suggest WFH has an effect on attrition.

**Q4.3** Calculate number of observations per week by group, and plot the results. For clarity, we'll focus on the experiment period, where `week` $\ge 0$. We've provided most of the code for you, you just need to fill in a few blanks.

In [ ]:
treatment_count = weekly.where(...).where('week', are.above_or_equal_to(0)).select('week', 'performance').group('...', collect = np.count_nonzero)

control_count = weekly.where(...).where('week', are.above_or_equal_to(0)).select('week', 'performance').group('...', collect = np.count_nonzero)

plt.plot(treatment_count.column('...'), treatment_count.column('performance count_nonzero'), label = 'Treatment')
plt.plot(control_count.column('...'), control_count.column('performance count_nonzero'), label = 'Control')
plt.title('Number of Working Employees Over Time')
plt.xlabel('Weeks Relative to Experiment Start Date')
plt.legend();

In [ ]:
_ = ok.grade("q4_3")

We can measure this even more precisely using quit data.

Run the cell below to load the quit data below.

In [ ]:
#run this cell
quits = Table.read_table('wfh_quits.csv')
quits.show(5)

These are data on whether an employee has quit by a given week. The column `quitjob` is an indicator for whether an employee as quit the job by a given week. For example, for an employee that quits in week 10, `quitjob = 1` for `week = 10`, `week = 11`, `week = 12`, and so on.

Note: The individual identifier (`anonid`) has been scrambled in this table, so we can't join these to other tables.

**Q4.4:** Plot quit rates by group over time. Use the code for the last two figures as references.

In [ ]:
#as above, create tables with average outcomes (this time, quitjob) by week, separately for treatment and control groups

treatment_quits = ...
control_quits = ...

#generate bar plot
plt.plot(treatment_quits.column('week'), treatment_quits.column('quitjob mean'), label = 'Treatment')
plt.plot(control_quits.column('week'), control_quits.column('quitjob mean'), label = 'Control')
plt.title('Quits Over Time')
plt.xlabel('Weeks Relative to Experiment Start Date')
plt.legend();

In [ ]:
_ = ok.grade("q4_4")

**Q4.5** What pattern do you notice? How do quit rates compare, and how does that change over time?

*write your answer here*

There is certainly cause for concern here. By the end of the experiment, a large number of employees have left the company, and therefore left our dataset. Moreover, the quit rate is particularly large for the control group.

----
## Section 5: Who Quits? Or, Can We Sign the Attrition Bias? <a id='bias'></a>

WFH clearly reduces employee turnover. This is a positive outcome for Ctrip--attrition is costly! But it also means we have an attrition problem, which complicates the experiment analysis. How do we estimate the causal effect of WFH on employee productivity if so many control employees have quit by the end of the experiment?

There's no perfect fix for this. In some experiments, we can try to reduce attrition by making sure to track participants as best we can. But in this case, it's not clear how we would do that--these employees have left the company, so how would we continue to track them? And what would we measure anyway?

However, we can at least try to figure out the *sign* of attrition bias by looking at who decides to leave. That is, we can try to figure out whether the bias is *positive*, so that our estimate is more positive than the true average causal effect, or *negative*, so that our estimate is more negative than the true average causal effect. In particular, we want to know who is more likely to leave: the more productive employees, or the less productive employees?

To do this, we'll take data on who quits and join that with our initial table, `experiment`. Then we'll compare the characteristics of employees that quit to those that don't. Then we'll have a sense for what type of worker is more likely to 'leave' the data.

First, run the cell below to load the data that identifies which employees quit during the experiment.

In [ ]:
#run this cell
everquit = Table.read_table('wfh_everquit.csv')
everquit.show(5)

There is one row for each `personid`. The column `quitjob` is an indicator for whether a given person quits or leaves the job during the experiment.

**Q5.1** Join with the table we created earlier `experiment`. (Hint: see [here](https://www.inferentialthinking.com/chapters/08/4/Joining_Tables_by_Columns) for a Data 8 referesher on table joins.)

In [ ]:
# we'll rewrite over experiment_clean with this new table
experiment_combined = ...
experiment_combined.show(5)

In [ ]:
_ = ok.grade("q5_1")

**Q5.2** Pooling both the treatment and control groups, compare the characteristics for employees that quit to employees that do not. What are some key differences you notice? [We are looking for a comparison similar to the comparison made in **Q3.1**.]

In [ ]:
# write code here

*describe comparison here*

**Q5.3** Given the differences you observe, should we be concerned about attrition bias? If you think it is a concern, what is the *sign* of the bias you expect? Will our estimate be more *positive* or more *negative* than the true causal effect of WFH on worker productivity? Why?

*write your answer here*

We'll discuss this question more as a group in class.

----
Congratulations, you've finished Lab 2! To submit the lab, run the two cells below:

In [ ]:
# For your convenience, you can run this cell to run all the tests at once
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
_ = ok.submit()